# Interactive Map

This Jupyter Notebook facilitates the investigation of transport routes for wind farm construction. An interactive map html is outputted to the output_files folder which displays the transport network across Northern Ireland with icons representing the harbours and airport (with popups containing their website addresses). The proposed site locations are also displayed with popups containing the results of the proximity and site characteristics analyses.

There is an optional section at the end of the Jupyter Notebook to create an additional interactive map output html that, alongside the transport information, contains a streamlined proximity and site characteristic results popup.

**Warning!** - This Notebook can only be utilized after the proximity analysis (Proximity_analysis.py) and site characteristics analysis (Site_characteristics.py) have been undertaken as the outputs from these analyses are used as inputs for the interactive map. The inputs and outputs of this notebook are outlined below. 

***Inputs: <br>***
- data_files/Counties.shp <br>
- data_files/Site_Locations.shp <br>
- output_files/Proximity_analysis.csv <br>
- output_files/Site_characteristics.csv <br>
- data_files/Airports.csv <br>
- data_files/Harbours.csv <br>

***Outputs: <br>***
- output_files/Site_analysis_results_with_transport.html <br>
- output_files/Site_analysis_results_with_transport_streamlined.html *(optional)*

## The Procedure
The interactive maps are created by running the code cells below. Notes describing what each section of code does can be found above each code cell (and in some cases within the code as comments).

Running the cell below will import **pandas**, **geopandas**, **folium** and **matplotlib.colors**, all of which are required in the code below. 

*Note:* We will import **matplotlib.colors** under an alias of **colors**.

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.colors as colors

The interactive map will be created using **folium**. Firstly, we want to create a location to zoom the interactive map to when it is initially opened by finding the centre of the extent of the input data (study extent).

To do this we will use the extent of the counties shapefile here as our study extent. The counties shapefile is loaded using **geopanda**'s .read_file() function which will create a GeoDataFrame called **counties**.

The starting coordinates to pass to **folium** should be in latitude/longitude format so we will set the CRS of the counties layer to the EPSG code for WGS84 lat/lon here (4326).

In [2]:
counties = gpd.read_file('data_files/Counties.shp')
counties.to_crs(epsg=4326, inplace=True)

The total bounds of the counties data are used to find max and min x and y values for the study extent. The starting coordinates are then calculated by finding the midpoint of the x values and midpoint of the y values.

*Note:* The centre (starting coordinates) are formatted as [y, x] as y represents the latitude and x represents the longitude.

In [3]:
xmin, ymin, xmax, ymax = counties.total_bounds
# the x midpoint is calulted by dividing the range in x values by 2 and adding this to the x min value
x = xmin+((xmax-xmin)/2)
# the y midpoint is calulted by dividing the range in y values by 2 and adding this to the y min value
y = ymin+((ymax-ymin)/2)
centre = [y, x]

We will now create our **folium** map by passing our starting coordinates and an initial zoom to **folium**’s Map() function. We will call our map **m** and display it here.

If you are using this notebook for a study in a different area please feel free to experiment with the initial zoom value here by re-running the cell and increasing/decreasing the zoom_start value. *Note:* The zoom_start value should be an integer.

In [4]:
m = folium.Map(centre, zoom_start=9)
m

The sites shapefile is loaded using the **geopanda**’s .read_file() function which will create a GeoDataFrame called **sites**.

In [5]:
sites = gpd.read_file('data_files/Site_Locations.shp')

We will rename the **sites** GeoDataFrame naming column label from **Name** to **Site Name** here to ensure that the naming column label will match those of the results CSVs containing the site information.

In [6]:
sites = sites.rename(columns={'Name': 'Site Name'})

The proximity analysis and site characteristics results CSVs are loaded using **panda**’s read_csv() function creating two DataFrames called **proximity_info** and **characteristics_info**.

In [7]:
proximity_info = pd.read_csv('output_files/Proximity_analysis.csv')
characteristics_info = pd.read_csv('output_files/Site_characteristics.csv')

The **proximity_info** and **characteristics_info** DataFrames are merged onto the **sites** GeoDataFrame based on the common column label **Site Name** creating a new GeoDataFrame called **sites_with_info** which contains all the site analysis results.

In [8]:
sites_with_info = sites.merge(proximity_info, on='Site Name')
sites_with_info = sites_with_info.merge(characteristics_info, on='Site Name')

A list of site colours is then created (here they match those used in the locator map).  

In [9]:
site_colors = ['orangered', 'olivedrab', 'dodgerblue']

A dictionary of arguments is created containing information on how we wish the **sites_with_info** to be displayed on our map.  

The site colours are added to the dictionary colormap **cmap** using the **colors**’s ListedColormap() function.

In [10]:
sites_args = {
    'm': m,  # add the polygons to the same map we created above
    'cmap': colors.ListedColormap(site_colors),  # color the polygons using the site colours list previously created
    'tooltip': False,  # remove tooltips (information when hovering)
    'popup': True  # show the site information as a popup when we click on the polygon
}

We now plot the **sites_with_info** polygons to our map **m** by using the **sites_args** with the ** unpacking operator.

*Note:* The site names will appear in a legend here.

In [11]:
sites_with_info.explore('Site Name', **sites_args)

To plot the airport and harbour locations two GeoDataFrames (**airports** and **harbours**) are created. The CVS files are loaded using **panda**’s read_csv() function creating DataFrames which are then converted to GeoDataFrames (with the required information columns) using **geopanda**’s GeoDataFrame() function.

In [12]:
dfa = pd.read_csv('data_files/Airports.csv')  # read the CSV data

# create a new GeoDataFrame
airports = gpd.GeoDataFrame(dfa[['Name', 'Website']],  # use the CSV data, but only the Name/Website columns
                            # set the geometry using points_from_xy()
                            geometry=gpd.points_from_xy(dfa['Longitude'], dfa['Latitude']), 
                            crs='epsg:4326')  # set the CRS using the EPSG code for WGS84 lat/lon

dfh = pd.read_csv('data_files/Harbours.csv')  # read the CSV data

# create a new GeoDataFrame
harbours = gpd.GeoDataFrame(dfh[['Name', 'Website']],  # use the CSV data, but only the Name/Website columns
                            # set the geometry using points_from_xy()
                            geometry=gpd.points_from_xy(dfh['Longitude'], dfh['Latitude']), 
                            crs='epsg:4326')  # set the CRS using the EPSG code for WGS84 lat/lon

A dictionary of arguments is created containing information on how we wish the **airports** to be displayed on our map.

*Note:* The marker style is passed to the **marker_kwds** as a dictionary.

In [13]:
airport_args = {
    'm': m,  # add the markers to the same map we created above
    'marker_type': 'marker',  # use a marker for the points, instead of a circle
    'tooltip': False,  # remove tooltips (information when hovering)
    'popup': True,  # show the airport information as a popup when we click on the marker
    'legend': False,  # don't show a separate legend for the airports
    # make the markers red with a plane icon from FA
    'marker_kwds': {'icon': folium.Icon(color='red', icon='plane', prefix='fa')}
}

A dictionary of arguments is created containing information on how we wish the **harbours** to be displayed on our map.

*Note:* The marker style is passed to the **marker_kwds** as a dictionary.

In [14]:
harbour_args = {
    'm': m,  # add the markers to the same map we created above
    'marker_type': 'marker',  # use a marker for the points, instead of a circle
    'tooltip': False,  # remove tooltips (information when hovering)
    'popup': True,  # show the harbour information as a popup when we click on the marker
    'legend': False,  # don't show a separate legend for the harbours
    # make the markers green with a ship icon from FA
    'marker_kwds': {'icon': folium.Icon(color='green', icon='ship', prefix='fa')} 
}

We now plot the **airports** and **harbours** to our map **m** by using the **airports_args** and **harbours_args** with the ** unpacking operator.

In [15]:
airports.explore('Name', **airport_args)
harbours.explore('Name', **harbour_args)

The map **m** is now saved to a html called **Site_analysis_results_with_transport**.

In [16]:
m.save('output_files/Site_analysis_results_with_transport.html')

#### *The following section is optional if you wish to create a more streamlined popup for the **sites**.*

First, we will create a new **folium** map using the same starting coordinates and zoom as our previous map **m** by passing these values to **folium**’s Map() function. We will call the streamlined map **ms**.

In [17]:
ms = folium.Map(centre, zoom_start=9)

A streamlined **sites_with_info** GeoDataFrame called **sites_info_streamlined** is created by creating a copy of the **sites_with_info** GeoDataFrame but with only the required columns we wish to include in the streamlined version.

**Warning!** - The geometry column of the GeoDataFrame must be copied.

In [18]:
sites_info_streamlined = sites_with_info[['geometry','Site Name', 'DistNearestAONB']].copy()

We now need to update the data display dictionaries we previously defined so that the data will display on our new map **ms**. We will do this by creating a dictionary update with the map key and new **ms** value and passing this to **python**’s dictionary.update() function.

In [19]:
dict_update = {'m': ms}  # define the dictionary update
sites_args.update(dict_update)  # pass the update to the sites display arguments
airport_args.update(dict_update)  # pass the update to the airports display arguments
harbour_args.update(dict_update)  # pass the update to the harbours display arguments

We now plot the **sites_info_streamlined** polygons, **airports** and **harbours** to our streamlined map **ms** by using the updated **sites_args**, **airports_args** and **harbour_args** respectively with the ** unpacking operator.

*Note:* The site names will appear in a legend here.

In [20]:
sites_info_streamlined.explore('Site Name', **sites_args)
airports.explore('Name', **airport_args)
harbours.explore('Name', **harbour_args)

The streamlined map **ms** is now saved to a different html called **Site_analysis_results_with_transport_streamlined**.

In [21]:
ms.save('output_files/Site_analysis_results_with_transport_streamlined.html')

#### The interactive map htmls can now be found in the output_files folder.